In [1]:
import sys
sys.executable

'/opt/homebrew/Caskroom/miniconda/base/envs/bioenv/bin/python'

In [ ]:
!pip install datasets
!pip install pyabf
!pip install wfdb
!pip install scipy
!pip install json

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import spacy
import datasets
from datasets import Dataset
import tqdm
#import evaluate
from collections import Counter, defaultdict
import pyabf
import ast
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import json
import wfdb
import scipy.io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
folder_path = "/content/drive/MyDrive/physionet.org"

In [2]:
# Reading the .hea file
record = wfdb.rdheader(folder_path + '/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001')

# Display the header information
print(record)

# Reading the .mat file
mat_data = scipy.io.loadmat(folder_path + '/files/challenge-2020/1.0.2/training/cpsc_2018/g1/A0001.mat')

# Display the .mat file content
print(mat_data)


{'val': array([[  28,   39,   45, ...,  258,  259,  259],
       [   7,   11,   15, ...,  248,  249,  250],
       [ -21,  -28,  -30, ...,  -10,  -10,   -9],
       ...,
       [-112, -110, -108, ...,  194,  194,  195],
       [-596, -590, -582, ...,  307,  307,  307],
       [ -16,   -7,    2, ...,  213,  214,  214]], dtype=int16)}


In [ ]:

# Path to the SHA256SUMS.txt file
sha256_file_path = folder_path + '/files/challenge-2020/1.0.2/SHA256SUMS.txt'

data = []
error_files = []

# Read the SHA256SUMS.txt file
with open(sha256_file_path, 'r') as file:
    lines = file.readlines()

for line in lines:
    parts = line.split()
    file_path = parts[1] if len(parts) > 1 else None

    if file_path and file_path.startswith('training') and file_path.endswith('.hea'):
        base_file_path = folder_path + '/files/challenge-2020/1.0.2/' + file_path[:-4]

        try:
            # Read the .hea file
            hea_record = wfdb.rdheader(base_file_path)

            # Read the .mat file
            mat_file_path = base_file_path + ".mat"
            mat_data = scipy.io.loadmat(mat_file_path)

            # Extract data and diagnosis
            record_dict = hea_record.__dict__
            comments = record_dict['comments']
            dx_comment = next((comment for comment in comments if comment.startswith('Dx:')), None)
            diagnosis = dx_comment.split('Dx: ')[1] if dx_comment else "Unknown"
            input_data = mat_data['val'].flatten().tolist()

            # Create the dictionary and add to the list
            record_dict = {
                "input": input_data,
                "output": diagnosis
            }
            data.append(record_dict)

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
            error_files.append(base_file_path)

        # Save the data to a file every 1000 records
        if len(data) % 1000 == 0:
            output_file_path = f'physionet_dataset_{len(data)}.json'

            with open(output_file_path, 'w') as f:
                json.dump(data, f)

            print(f"data_list has been saved to {output_file_path}")

# Optionally, save any remaining data after the loop ends
if data:
    output_file_path = '/content/drive/MyDrive/physionet_dataset_final.json'
    with open(output_file_path, 'w') as f:
        json.dump(data, f)

    print(f"Final data_list has been saved to {output_file_path}")
